# Beyond Basic Cleaning: Preparing Data for ML

*Step-by-step notebook to accompany the YouTube video.*

## 1) Load the dataset
We'll start with a deliberately **messy** dataset that contains missing values, outliers, and inconsistent date formats.

In [3]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('ml_prep_demo_raw.csv')
df.head()

,id,event_date,age,income,gender,region,purchases,category,churn
0,1,2023-04-13,34.0,NaN,Female,East,2.0,D,0
1,2,2023-12-15,40.0,54770.0,Male,East,2.0,NaN,1
2,3,2023-09-28,NaN,85399.0,Female,East,2.0,B,0
3,4,2023-04-17,44.0,52703.0,Male,NaN,NaN,B,1
4,5,2023-03-13,NaN,53504.0,NaN,North,7.0,B,0


## 2) Quick EDA & data types
Check shape, nulls, and dtypes to see what we’re dealing with.

In [5]:
print(df.shape)
df.info()
df.isna().mean().sort_values(ascending=False)

(1500, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1500 non-null   int64  
 1   event_date  1500 non-null   object 
 2   age         1405 non-null   float64
 3   income      1396 non-null   float64
 4   gender      1379 non-null   object 
 5   region      1414 non-null   object 
 6   purchases   1397 non-null   float64
 7   category    1396 non-null   object 
 8   churn       1500 non-null   int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 105.6+ KB


gender        0.080667
income        0.069333
category      0.069333
purchases     0.068667
age           0.063333
region        0.057333
id            0.000000
event_date    0.000000
churn         0.000000
dtype: float64

## 3) Fix dates & basic schema
The `event_date` has mixed formats. We'll coerce to datetime safely; invalid ones become `NaT`.

In [16]:
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df[['event_date']].head()

,event_date
0,2023-04-13
1,2023-12-15
2,2023-09-28
3,2023-04-17
4,2023-03-13


df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df[['event_date']].head()

## 4) Feature engineering from dates
Create features like `event_month`, `event_dayofweek`, and `recency_days` (relative to dataset max date).

In [18]:
max_date = df['event_date'].max()
df['event_month'] = df['event_date'].dt.month
df['event_dayofweek'] = df['event_date'].dt.dayofweek
df['recency_days'] = (max_date - df['event_date']).dt.days
df[['event_date','event_month','event_dayofweek','recency_days']].head()

,event_date,event_month,event_dayofweek,recency_days
0,2023-04-13,4.0,3.0,297.0
1,2023-12-15,12.0,4.0,51.0
2,2023-09-28,9.0,3.0,129.0
3,2023-04-17,4.0,0.0,293.0
4,2023-03-13,3.0,0.0,328.0


## 5) Split features & target
We'll predict `churn` as the target. Identify numeric vs categorical columns.

In [20]:
target = 'churn'
feature_cols = [c for c in df.columns if c not in [target]]
X = df[feature_cols].copy()
y = df[target].copy()

numeric_features = ['age','income','purchases','event_month','event_dayofweek','recency_days']
categorical_features = ['gender','region','category']
date_cols = ['event_date']  # will drop raw date after deriving features

X = X.drop(columns=date_cols)
X.head()

,id,age,income,gender,region,purchases,category,event_month,event_dayofweek,recency_days
0,1,34.0,NaN,Female,East,2.0,D,4.0,3.0,297.0
1,2,40.0,54770.0,Male,East,2.0,NaN,12.0,4.0,51.0
2,3,NaN,85399.0,Female,East,2.0,B,9.0,3.0,129.0
3,4,44.0,52703.0,Male,NaN,NaN,B,4.0,0.0,293.0
4,5,NaN,53504.0,NaN,North,7.0,B,3.0,0.0,328.0


## 6) Outlier exploration (numeric)
Use Z-score/IQR to **detect** outliers before deciding to cap/winsorize.

In [22]:
from scipy import stats
import numpy as np

numX = X[numeric_features].copy()
z = np.abs(stats.zscore(numX, nan_policy='omit'))
outlier_mask = (z > 3).any(axis=1)
outlier_rate = outlier_mask.mean()
outlier_rate

0.012

## 7) Preprocessing pipeline
- **Impute** missing values (KNN for numeric; constant for categorical)
- **Scale** numeric features
- **One-hot encode** categoricals

We’ll wrap everything in a scikit-learn **Pipeline** to avoid leakage and ensure repeatability.

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=3)),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

### Optional: Outlier capping (winsorization)
We can add a custom transformer to cap extreme values after imputation.

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin

class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, quantile_low=0.01, quantile_high=0.99):
        self.quantile_low = quantile_low
        self.quantile_high = quantile_high
        self.lows_ = None
        self.highs_ = None

    def fit(self, X, y=None):
        import pandas as pd
        X = pd.DataFrame(X)
        self.lows_ = X.quantile(self.quantile_low)
        self.highs_ = X.quantile(self.quantile_high)
        return self

    def transform(self, X):
        import pandas as pd
        X = pd.DataFrame(X).clip(lower=self.lows_, upper=self.highs_, axis=1)
        return X.values


## 8) Train a model with the preprocessing pipeline
We’ll use Logistic Regression as a baseline to demonstrate how preprocessing and modeling fit together.

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

clf = Pipeline(steps=[('preprocess', preprocessor), ('model', LogisticRegression(max_iter=200))])
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)[:,1]

acc = accuracy_score(y_test, preds)
auc = roc_auc_score(y_test, probs)

print("Accuracy:", round(acc, 4))
print("ROC AUC:", round(auc, 4))
print("\nClassification Report:\n", classification_report(y_test, preds))

Accuracy: 0.6833
ROC AUC: 0.5135

Classification Report:
               precision    recall  f1-score   support

           0       0.68      1.00      0.81       205
           1       0.00      0.00      0.00        95

    accuracy                           0.68       300
   macro avg       0.34      0.50      0.41       300
weighted avg       0.47      0.68      0.55       300



### Compare with winsorization variant

In [32]:
clf_w = Pipeline(steps=[('preprocess', preprocessor_w), ('model', LogisticRegression(max_iter=200))])
clf_w.fit(X_train, y_train)
preds_w = clf_w.predict(X_test)
probs_w = clf_w.predict_proba(X_test)[:,1]
acc_w = accuracy_score(y_test, preds_w)
auc_w = roc_auc_score(y_test, probs_w)

print("Winsorized Accuracy:", round(acc_w, 4))
print("Winsorized ROC AUC:", round(auc_w, 4))

Winsorized Accuracy: 0.6833
Winsorized ROC AUC: 0.5368


## 9) Export the preprocessing pipeline
Save the fitted preprocessor + model for reuse.

In [34]:
import joblib
joblib.dump(clf, 'ml_prep_pipeline_baseline.joblib')
joblib.dump(clf_w, 'ml_prep_pipeline_winsor.joblib')
'Artifacts saved'

'Artifacts saved'